In [2]:
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer

Find vocab size of the dataset

https://github.com/huggingface/transformers/issues/4151

https://stackoverflow.com/questions/73204460/how-to-pretrain-bart-using-custom-datasetnot-fine-tuning

In [3]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART", 
                                          do_lower_case=False, use_fast=False, keep_accents=True)

model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")

In [4]:
bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")

In [5]:
inp = tokenizer("I am a boy </s> <2en>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids 

out = tokenizer("<2hi> मैं  एक लड़का हूँ </s>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids

In [6]:
print(inp)

tensor([[  466,  1981,    80, 25573, 64001, 64004]])


In [7]:
print(out)

tensor([[64006,   942,    43, 32720,  8384, 64001]])


In [8]:
print(out[:,0:-1])

tensor([[64006,   942,    43, 32720,  8384]])


In [9]:
print(out[:,1:])

tensor([[  942,    43, 32720,  8384, 64001]])


In [10]:
model_outputs=model(input_ids=inp, decoder_input_ids=out[:,0:-1], labels=out[:,1:])


In [11]:
model_outputs.loss

tensor(8.7369, grad_fn=<NllLossBackward0>)

In [18]:
model_outputs.logits
#model.eval()

tensor([[[ 0.5566,  0.2969,  0.5566,  ...,  0.7106, -0.0018, -0.9501],
         [ 0.4918,  0.1316,  0.4917,  ...,  0.4986,  0.3004, -0.6960],
         [ 0.7462,  0.7578,  0.7462,  ...,  0.5706, -1.1140, -0.2669],
         [ 0.3260,  1.4900,  0.3260,  ..., -0.2245,  0.0107, -0.2596],
         [ 0.2783,  1.5880,  0.2783,  ...,  0.1308,  0.2887,  0.0781]]],
       grad_fn=<AddBackward0>)

In [13]:
model_output=model.generate(inp, use_cache=True, num_beams=4, max_length=20, min_length=1, 
                            early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, 
                            eos_token_id=eos_id, 
                            decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>"))

In [14]:
print(model_output)

tensor([[64004,   466,  1981,    80, 25573, 64001]])


In [15]:
decoded_output=tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)

2023-07-02 15:32:21.916176: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-02 15:32:21.916377: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-07-02 15:32:21.916388: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [16]:
print(decoded_output)

I am a boy
